In [ ]:
:build_env PKG_CONFIG_PATH=/home/jmagin/.local/opt/mambaforge/envs/cdshealpix/lib/pkgconfig
:dep cdshealpix
:dep rayon
:dep geographiclib-rs
:dep geodesy
:dep csv = { version = "*" }
:dep serde
:dep itertools
:dep plotters = { version = ">=0.3.6", default-features = false, features = ["evcxr", "all_series", "all_elements"] }

In [ ]:
use cdshealpix::compass_point::Cardinal;
use cdshealpix::nested;
use geodesy::{math::series::FourierCoefficients, Ellipsoid};
use geographiclib_rs::{Geodesic, PolygonArea, Winding};
use rayon::prelude::*;

use plotters::prelude::*;

In [ ]:
fn cell_area(boundary: Vec<(f64, f64)>, g: &Geodesic) -> f64 {
    let mut pa = PolygonArea::new(g, Winding::CounterClockwise);

    for (lon, lat) in boundary {
        pa.add_point(lat, lon);
    }

    let (_, area, _) = pa.compute(false);

    area
}

In [ ]:
fn compute_boundary(cell_id: u64, layer: &nested::Layer, step: u32) -> Vec<(f64, f64)> {
    nested::path_along_cell_edge(layer.depth(), cell_id, &Cardinal::S, false, step)
        .into_iter()
        .map(|p| (p.0.to_degrees(), p.1.to_degrees()))
        .collect::<Vec<(f64, f64)>>()
}

In [ ]:
fn authalic_to_geographic(
    boundary: Vec<(f64, f64)>,
    ellipsoid: &Ellipsoid,
    coefficients: &FourierCoefficients,
) -> Vec<(f64, f64)> {
    boundary
        .into_iter()
        .map(|p| {
            (
                p.0,
                ellipsoid
                    .latitude_authalic_to_geographic(p.1.to_radians(), coefficients)
                    .to_degrees(),
            )
        })
        .collect::<Vec<(f64, f64)>>()
}

In [ ]:
let wgs84 = Geodesic::wgs84();
let sphere = Geodesic::new(6371000f64, 0f64);

let ellipsoid = Ellipsoid::named("WGS84").unwrap();
let coefficients = ellipsoid.coefficients_for_authalic_latitude_computations();

In [ ]:
let layer = nested::get(12);
let step = 30;

let cell_ids = (0..layer.n_hash()).collect::<Vec<u64>>();
let areas_sphere = cell_ids.par_iter().map(|&hash| compute_boundary(hash, layer, step)).map(|boundary| cell_area(boundary, &sphere)).collect::<Vec<f64>>();
let areas_wgs84 = cell_ids.par_iter().map(|&hash| compute_boundary(hash, layer, step)).map(|boundary| cell_area(boundary, &wgs84)).collect::<Vec<f64>>();

let areas_wgs84_authalic = cell_ids.par_iter()
    .map(|&hash| compute_boundary(hash, layer, step))
    .map(|boundary| authalic_to_geographic(boundary, &ellipsoid, &coefficients))
    .map(|boundary| cell_area(boundary, &wgs84))
    .collect::<Vec<f64>>();

let n_cells = layer.n_hash() as f64;
let theoretical_area_sphere = sphere.area() / n_cells;
let theoretical_area_wgs84 = sphere.area() / n_cells;

// let ratio_sphere = areas_sphere.par_iter().map(|&area| area / theoretical_area_sphere).collect::<Vec<f64>>();
// let ratio_wgs84 = areas_wgs84.par_iter().map(|&area| area / theoretical_area_wgs84).collect::<Vec<f64>>();
// let ratio_wgs84_authalic = areas_wgs84_authalic.par_iter().map(|&area| area / theoretical_area_wgs84).collect::<Vec<f64>>();

In [ ]:
use csv::Writer;
use itertools::izip;

#[derive(serde::Serialize)]
struct Row {
    cell_id: u64,
    area_sphere: f64,
    area_naive_wgs84: f64,
    area_wgs84: f64,
}

let mut wtr = Writer::from_path(format!("cell-areas-depth-{}.csv", layer.depth()))?;
for (cell_id, area_sphere, area_naive_wgs84, area_wgs84) in izip!(
    cell_ids.iter().cloned(),
    areas_sphere.iter().cloned(),
    areas_wgs84.iter().cloned(),
    areas_wgs84_authalic.iter().cloned(),
) {
    wtr.serialize(Row {cell_id, area_sphere, area_naive_wgs84, area_wgs84 }).unwrap();
    wtr.flush().unwrap();
}